In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

W0728 10:58:18.745058 13360 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [2]:
#!pip install bert-tensorflow

In [3]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [4]:
OUTPUT_DIR = '.\data_out'
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: .\data_out *****


In [5]:
'''
NOT NEEDED
import csv
def data_from_csv(input_file):
    file_CSV = open(input_file)
    data_CSV = csv.reader(file_CSV)
    list_CSV = list(data_CSV)
    reader = csv.reader(input_file,)
    lines = []; words = []; labels = [];
    for row in list_CSV[1:]:
        if (row[0]=='.'):
            l = ' '.join([label for label in labels if len(label) > 0])
            w = ' '.join([word for word in words if len(word) > 0])
            words=[]
            labels = []
            lines.append((w, l))
        else:
            words.append(row[0])
            labels.append(row[1])
    return lines
    '''


"\nNOT NEEDED\nimport csv\ndef data_from_csv(input_file):\n    file_CSV = open(input_file)\n    data_CSV = csv.reader(file_CSV)\n    list_CSV = list(data_CSV)\n    reader = csv.reader(input_file,)\n    lines = []; words = []; labels = [];\n    for row in list_CSV[1:]:\n        if (row[0]=='.'):\n            l = ' '.join([label for label in labels if len(label) > 0])\n            w = ' '.join([word for word in words if len(word) > 0])\n            words=[]\n            labels = []\n            lines.append((w, l))\n        else:\n            words.append(row[0])\n            labels.append(row[1])\n    return lines\n    "

In [6]:
#train = data_from_csv('./NER/train.csv')
#dev = data_from_csv('./NER/dev.csv')
#test = data_from_csv('./NER/test.csv')

In [8]:
import pandas
train = pandas.read_csv('./NER/train.csv')
train = train.iloc[1:4988,:]

test = pandas.read_csv('./NER/dev.csv')
test = test.iloc[1:4968,:]

In [13]:
import csv
def make_sentences(input_file):
    with open(input_file, 'r') as f:
                rf = list(csv.reader(f))
                lines = [];words = [];labels = []
                for line in rf[1:]:
                    word = line[0]
                    label = line[1]
                    # here we dont do "DOCSTART" check
                    if len(word)<=1 and word[-1] == '.':
                        l = ' '.join([label for label in labels if len(label) > 0])
                        w = ' '.join([word for word in words if len(word) > 0])
                        lines.append((w,l))
                        words=[]
                        labels = []
                    else:
                        words.append(word)
                        labels.append(label)
    return lines
train = make_sentences('./NER/train.csv')
train = pd.DataFrame(train)

In [14]:
print(train)

                                                       0  \
0      113416550 PRGH 13523357 630190 6/7 1999 12:00:...   
1      He underwent CT and ERCP at the Lisonatemi Fay...   
2      He underwent an ECHO and endoscopy at Ingree a...   
3      He was found to have a large , bulging , extri...   
4      Fine needle aspiration showed atypical cells ,...   
5      Abdominal CT on April 14 , showed a 12 x 8 x 8...   
6      He denied any nausea , vomiting , anorexia , o...   
7      He states that his color in urine or in stool ...   
8      PAST MEDICAL HISTORY : He has hypertension and...   
9      PAST SURGICAL HISTORY : Status post left kidne...   
10            ALLERGIES : He has no known drug allergies   
11     MEDICATIONS PRIOR TO ADMISSION : Hydrochloroth...   
12     He had an uncomplicated postoperative course a...   
13     Advanced his diet on postop day # 4 to a trans...   
14     His PCA was discontinued on postop day # 4 , a...   
15                  PHYSICAL EXAMINATION

In [15]:
data_col = 0
label_col = 1

In [16]:
label_list = "[PAD]", "PATIENT", "DATE", "HOSPITAL", "PHONE", "DOCTOR", "LOCATION", "AGE", "ID", "NONE", "X", "[CLS]","[SEP]"

In [17]:
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[data_col], 
                                                                   text_b = None, 
                                                                   label = x[label_col]), axis = 1)

In [18]:
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

Instructions for updating:
Colocations handled automatically by placer.


W0728 11:03:19.973137 13360 deprecation.py:323] From C:\Users\rebec\Anaconda3\lib\site-packages\tensorflow\python\ops\control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0728 11:03:22.898733 13360 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [26]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import os
import pickle
from absl import flags,logging
import tensorflow as tf
import numpy as np
import csv

In [28]:
import logging
def filed_based_convert_examples_to_features(examples, label_list, max_seq_length, tokenizer, output_file,mode=None):
    writer = tf.python_io.TFRecordWriter(output_file)
    batch_tokens = []
    batch_labels = []
    for (ex_index, example) in enumerate(examples):
        if ex_index % 5000 == 0:
            logging.info("Writing example %d of %d" % (ex_index, len(examples)))  
        feature,ntokens,label_ids = convert_single_example(ex_index, example, label_list, max_seq_length, tokenizer, mode)
        batch_tokens.extend(ntokens)
        batch_labels.extend(label_ids)
        def create_int_feature(values):
            f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
            return f

        features = collections.OrderedDict()
        features["input_ids"] = create_int_feature(feature.input_ids)
        features["mask"] = create_int_feature(feature.mask)
        features["segment_ids"] = create_int_feature(feature.segment_ids)
        features["label_ids"] = create_int_feature(feature.label_ids)
        tf_example = tf.train.Example(features=tf.train.Features(feature=features))
        writer.write(tf_example.SerializeToString())
    # sentence token in each batch
    writer.close()
    return batch_tokens,batch_labels

In [31]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
output_file = "output_file.txt"
# Convert our train and test features to InputFeatures that BERT understands.
train_features = filed_based_convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer, output_file)

AttributeError: 'InputExample' object has no attribute 'text'

In [32]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["sequence_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [ ]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [ ]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [ ]:
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [ ]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

In [ ]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [ ]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)